<a href="https://colab.research.google.com/github/antonionipo/Data-analysis-for-populism-in-presidential-speeches/blob/main/data_analysis_for_populism_in_presidential_speeches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd
import re
from nltk import word_tokenize, download
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

## Preparação dos dados

In [99]:
# Leitura da base
df = pd.read_csv('populismo_final.csv', encoding='latin-1')
df.head()

,id,ordem,pres_nick_name,pres_label,date,frases,n.carac,nwords,povo,elite,potencial,mauricio_class,davi_class,paolo_class,discordancia,populismo
0,Sarney-I-1985-04-21-0006,210,José Sarney,Sarney-I,1985-04-21,Deus é testemunha de que eu*lhe entregaria o m...,132,23,0,0,0,0,0.0,0,0.0,0
1,Sarney-I-1985-04-21-0006,211,José Sarney,Sarney-I,1985-04-21,Eterniza-se com ele a legenda do idealismo que...,167,30,0,1,0,0,0.0,0,0.0,0
2,Sarney-I-1985-04-21-0006,212,José Sarney,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",196,34,1,1,1,0,0.0,0,0.0,0
3,Sarney-I-1985-04-21-0006,213,José Sarney,Sarney-I,1985-04-21,"Lágrimas temos todos, das fronteiras escondida...",123,21,0,1,0,0,0.0,0,0.0,0
4,Sarney-I-1985-04-21-0006,214,José Sarney,Sarney-I,1985-04-21,"Aqui estou, meus compatriotas, sob o peso de u...",84,16,0,0,0,0,0.0,0,0.0,0


### Preparação

In [100]:
# Cópia do dataframe
populista = df.copy()

In [101]:
# Returar colunas irrelevantes
populista.drop(['id', 'ordem', 'pres_nick_name', 'n.carac', 'nwords', 'povo', 'elite', 'mauricio_class', 'davi_class', 'paolo_class'], axis=1, inplace=True)
populista.head()

,pres_label,date,frases,potencial,discordancia,populismo
0,Sarney-I,1985-04-21,Deus é testemunha de que eu*lhe entregaria o m...,0,0.0,0
1,Sarney-I,1985-04-21,Eterniza-se com ele a legenda do idealismo que...,0,0.0,0
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",1,0.0,0
3,Sarney-I,1985-04-21,"Lágrimas temos todos, das fronteiras escondida...",0,0.0,0
4,Sarney-I,1985-04-21,"Aqui estou, meus compatriotas, sob o peso de u...",0,0.0,0


In [102]:
populista.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481198 entries, 0 to 481197
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   pres_label    481198 non-null  object 
 1   date          481198 non-null  object 
 2   frases        481198 non-null  object 
 3   potencial     481198 non-null  int64  
 4   discordancia  475395 non-null  float64
 5   populismo     481198 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 22.0+ MB


In [103]:
# Convert the 'date' column to datetime objects
populista['date'] = pd.to_datetime(populista['date'], format='%Y-%m-%d')

# Convert the 'potencial', 'discordancia', and 'populismo' to Boolean
populista['potencial'] = populista['potencial'].astype(bool)
populista['discordancia'] = (populista['discordancia']).astype(bool)
populista['populismo'] = (populista['populismo']).astype(bool)

# Convert 'frases' and 'pres_label' to string
populista['frases'] = populista['frases'].astype(str)
populista['pres_label'] = populista['pres_label'].astype(str)

# Display info to verify the change
populista.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481198 entries, 0 to 481197
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   pres_label    481198 non-null  object        
 1   date          481198 non-null  datetime64[ns]
 2   frases        481198 non-null  object        
 3   potencial     481198 non-null  bool          
 4   discordancia  481198 non-null  bool          
 5   populismo     481198 non-null  bool          
dtypes: bool(3), datetime64[ns](1), object(2)
memory usage: 12.4+ MB


In [104]:
populista.head()

,pres_label,date,frases,potencial,discordancia,populismo
0,Sarney-I,1985-04-21,Deus é testemunha de que eu*lhe entregaria o m...,False,False,False
1,Sarney-I,1985-04-21,Eterniza-se com ele a legenda do idealismo que...,False,False,False
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",True,False,False
3,Sarney-I,1985-04-21,"Lágrimas temos todos, das fronteiras escondida...",False,False,False
4,Sarney-I,1985-04-21,"Aqui estou, meus compatriotas, sob o peso de u...",False,False,False


In [105]:
# Verificar se há dado nulo
populista.isnull().sum()

,0
pres_label,0
date,0
frases,0
potencial,0
discordancia,0
populismo,0


In [106]:
# Filtrados os casos potencialmente populistas, retirada a coluna potencial
populista = populista.loc[populista['potencial'] == True]
populista.drop(columns=['potencial'], inplace=True)
populista.head()

<ipython-input-106-c4a1fb1b1f58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populista.drop(columns=['potencial'], inplace=True)


,pres_label,date,frases,discordancia,populismo
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",False,False
135,Sarney-I,1985-05-02,"Aqui estão educadores, estudantes, cientistas,...",False,False
446,Sarney-I,1985-05-18,"Na manhã de 15 de janeiro, há tão poucos e den...",False,False
469,Sarney-I,1985-05-18,"Antes, estávamos juntos para, em nome do povo,...",False,False
513,Sarney-I,1985-05-24,"O Nordeste é, sobretudo, um estado de espírito.",False,False


In [107]:
# Retirado e separado para posterior análise
y1 = populista['discordancia']
populista.drop(columns=['discordancia'], inplace=True)
populista.head()

<ipython-input-107-2fb667e526ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populista.drop(columns=['discordancia'], inplace=True)


,pres_label,date,frases,populismo
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",False
135,Sarney-I,1985-05-02,"Aqui estão educadores, estudantes, cientistas,...",False
446,Sarney-I,1985-05-18,"Na manhã de 15 de janeiro, há tão poucos e den...",False
469,Sarney-I,1985-05-18,"Antes, estávamos juntos para, em nome do povo,...",False
513,Sarney-I,1985-05-24,"O Nordeste é, sobretudo, um estado de espírito.",False


### Pré processamento

Padronziar formato das letras

In [108]:
populista['frases'] = populista['frases'].str.lower().tolist()
populista.head()

,pres_label,date,frases,populismo
2,Sarney-I,1985-04-21,"ninguém o excedeu no amor do povo, que acompan...",False
135,Sarney-I,1985-05-02,"aqui estão educadores, estudantes, cientistas,...",False
446,Sarney-I,1985-05-18,"na manhã de 15 de janeiro, há tão poucos e den...",False
469,Sarney-I,1985-05-18,"antes, estávamos juntos para, em nome do povo,...",False
513,Sarney-I,1985-05-24,"o nordeste é, sobretudo, um estado de espírito.",False


In [109]:
download('stopwords')
download('punkt_tab')
download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Definindo stop words, e caracteres especiais

In [110]:
stop_words_ptbr = stopwords.words('portuguese')
pattern = r'[^A-Za-zÀ-ÿ0-9\s]+'

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as']


Lematização e Stematização, remoção de stop words, de caracteres especiais, e vetorização-tokenização

In [113]:
class LemmaTokenizer:
    def __init__(self, regex_pattern='', stop_words=''):
        self.lemmatizer = WordNetLemmatizer()
        self.pattern = regex_pattern
        self.tokenize = lambda doc : word_tokenize(re.sub(self.pattern, '', doc))
        self.stop = stop_words
    def __call__(self, doc):
        return [self.lemmatizer.lemmatize(t) for t in self.tokenize(doc) if t not in self.stop]

In [114]:
lem = LemmaTokenizer(regex_pattern=pattern, stop_words=stop_words_ptbr)

In [115]:
vectorizer = CountVectorizer(tokenizer=lem)

In [116]:
# Mantendo a coluna original "frases" e criando uma nova coluna para a tokenização
populista['tokenized_frases'] = populista['frases'].apply(lambda x: lem(x))

# Visualizando para verificar
print(populista[['frases', 'tokenized_frases']].head())

                                                frases  \
2    ninguém o excedeu no amor do povo, que acompan...   
135  aqui estão educadores, estudantes, cientistas,...   
446  na manhã de 15 de janeiro, há tão poucos e den...   
469  antes, estávamos juntos para, em nome do povo,...   
513    o nordeste é, sobretudo, um estado de espírito.   

                                      tokenized_frases  
2    [ninguém, excedeu, amor, povo, acompanhou, lon...  
135  [aqui, educadores, estudantes, cientistas, rel...  
446  [manhã, 15, janeiro, tão, poucos, densos, mese...  
469  [ante, junto, nome, povo, conquistar, poder, s...  
513            [nordeste, sobretudo, estado, espírito]  


In [117]:
populista.head()

,pres_label,date,frases,populismo,tokenized_frases
2,Sarney-I,1985-04-21,"ninguém o excedeu no amor do povo, que acompan...",False,"[ninguém, excedeu, amor, povo, acompanhou, lon..."
135,Sarney-I,1985-05-02,"aqui estão educadores, estudantes, cientistas,...",False,"[aqui, educadores, estudantes, cientistas, rel..."
446,Sarney-I,1985-05-18,"na manhã de 15 de janeiro, há tão poucos e den...",False,"[manhã, 15, janeiro, tão, poucos, densos, mese..."
469,Sarney-I,1985-05-18,"antes, estávamos juntos para, em nome do povo,...",False,"[ante, junto, nome, povo, conquistar, poder, s..."
513,Sarney-I,1985-05-24,"o nordeste é, sobretudo, um estado de espírito.",False,"[nordeste, sobretudo, estado, espírito]"
